Una empresa de paquetería quiere repartir en una ciudad un conjunto de paquetes en distintas 
casas. Para ello quiere saber cual es la ruta más corta para repartir todos los paquetes. Un 
paquete tiene un peso y una prioridad de entrega (generados aleatoriamente) y un repartidor 
tiene una capacidad máxima de cuantos paquetes puede llevar al mismo tiempo, basado en 
el peso (generado aleatoriamente). <br>

Dado un grafo grande de casas (definido por el mismo equipo), donde los nodos representan 
una casa a donde se puede entregar un paquete, y existe una distancia entre cada nodo. 
Cuando el repartidor se quede sin paquetes que entregar debe regresar a la matriz de la 
empresa para verificar si existen paquetes por entregar y así hasta quedarse sin paquetes. El 
objetivo es empezar en el nodo de la empresa y minimizar el costo total de las rutas para 
entregar eficientemente los paquetes en el menor tiempo y con la menor distancia recorrida. 

Posibles datos de entrada: <br>
• Conjunto de paquetes con peso, prioridad y nodo de entrega.<br>
• Vehículo de entregas con peso máximo de paquetes que puede llevar.<br>
• Mapa de rutas que contiene el grafo de información.<br>

Objetivos:<br>
• Minimizar la distancia total recorrida por los vehículos.<br>
• Minimizar los tiempos de entrega de los paquetes.<br>
• Entregar todos los paquetes.<br>

Restricciones:<br>
• Los vehículos no deben superar su capacidad máxima<br>
• Todos los paquetes deben ser entregados<br>
• Los nodos del grafo no están todos interconectados, puedes necesitar ir a otra casa 
antes de llegar al objetivo.<br>

Sugerencias adicionales:<br>
• Empezar con un algoritmo genético aplicado a encontrar una ruta eficiente en un grafo
de ida y vuelta a varios puntos, y luego agregar las restricciones de paquetes y peso.<br>
• Probar diferentes configuraciones de parámetros para la configuración del genotipo, la 
evaluación de la solución, la generación de una nueva población o el factor de 
mutación.

In [155]:
# Lista de paquetes con prioridad y peso.
# El índice de la lista representa el número de paquete.
# Y cada paquete está conformado por [prioridad, peso]
ejemplo_listaPaquetes = [[8, 31],   #1
                 [10, 40],  #2
                 [2, 21],   #3
                 [5, 39],   #4
                 [7, 42],   #5
                 [1, 6],    #6
                 [3, 28],   #7
                 [9, 18],   #8
                 [4, 13],   #9
                 [6, 38]]   #10

# NOTA: Acorde la definición del problema, cada peso y prioridad de los paquetes tiene que ser
#       generado aleatoriamente. La lista anterior solo es un ejemplo.

# El problema no dice que el número de paquetes debe ser aleatorio,
# entonces podemos decir que siempre serán 10 paquetes

In [156]:
import math

# Diccionario de ciudades y sus coordenadas
# se tomó como referencia las coordenadas de este repositorio:
# https://github.com/hassanzadehmahdi/Romanian-problem-using-Astar-and-GBFS/blob/main/cities.txt

coordenadas = {
    "Arad": (29, 192),
    "Bucharest": (268, 55),
    "Craiova": (163, 22),
    "Drobeta": (91, 32),
    "Eforie": (420, 28),
    "Fagaras": (208, 157),
    "Giurgiu": (264, 8),
    "Hirsova": (396, 74),
    "Iasi": (347, 204),
    "Lugoj": (91, 98),
    "Mehadia": (93, 65),
    "Neamt": (290, 229),
    "Oradea": (62, 258),
    "Pitesti": (220, 88),
    "Rimnicu Vilcea": (147, 124),
    "Sibiu": (126, 164),
    "Timisoara": (32, 124),
    "Urziceni": (333, 74),
    "Vaslui": (376, 153),
    "Zerind": (44, 225)
}

In [157]:
# Función para calcular la distancia euclidiana (línea recta)
def distancia_euclidiana(punto1, punto2):
    distancia = math.sqrt((punto1[0] - punto2[0]) ** 2 + (punto1[1] - punto2[1]) ** 2)
    return round(distancia, 2) # redondear a dos decimales

In [158]:
# Ejemplo de uso
punto_a = coordenadas["Arad"]
punto_b = coordenadas["Sibiu"]
distancia = distancia_euclidiana(punto_a, punto_b)
print(f"La distancia euclidiana entre las dos es: {distancia:.2f}")

La distancia euclidiana entre las dos es: 100.96


In [159]:
#   Permite generar un diccionario con las distancias euclidianas entre
#   la ciudad base y el resto, que servirá como heurística en A*Star
def generarListaDistancias(ciudad_base):
    distancias = {}
    coordenada_base = coordenadas[ciudad_base]
    
    for ciudad, coordenada in coordenadas.items():
        if ciudad != ciudad_base:
            distancias[ciudad] = distancia_euclidiana(coordenada_base, coordenada)
    
    return distancias

In [160]:
# Ejemplo de uso
ejemplo_lista = generarListaDistancias("Arad")
print(ejemplo_lista)

{'Bucharest': 275.48, 'Craiova': 216.46, 'Drobeta': 171.59, 'Eforie': 424.0, 'Fagaras': 182.39, 'Giurgiu': 298.46, 'Hirsova': 385.5, 'Iasi': 318.23, 'Lugoj': 112.61, 'Mehadia': 142.21, 'Neamt': 263.61, 'Oradea': 73.79, 'Pitesti': 217.48, 'Rimnicu Vilcea': 136.19, 'Sibiu': 100.96, 'Timisoara': 68.07, 'Urziceni': 326.1, 'Vaslui': 349.18, 'Zerind': 36.25}


In [161]:
# Grafo completo de las ciudades, las mismas que se usaron en el problema de viajeros.
# En este caso está almacenado como diccionario.
# Se considerará Bucharest como el lugar de la empresa, ya que tenemos las distancias euclidianas de Bucharest
# a las demás ciudades.

# El grafo es un diccionario que contiene un nodo representando una ciudad, y cada nodo es un diccionario que almacena
# las ciudades a la que está conectada, que contiene la distancia entre ciudades.

# Grafo de distancias entre ciudades, sacadas usando la función distancia_euclidiana
grafo = {
    "Arad": {
        "Zerind": 36.25,    # Distancia entre Arad a Zerind.
        "Sibiu": 100.96,    # Distancia entre Arad a Sibiu.
        "Timisoara": 68.07  # Distancia entre Arad a Timisoara.
    },
    "Zerind": {
        "Oradea": 37.59,
        "Arad": 36.25
    },
    "Oradea": {
        "Sibiu": 113.72,
        "Zerind": 37.59
    },
    "Timisoara": {
        "Lugoj": 64.47,
        "Arad": 68.07
    },
    "Lugoj": {
        "Mehadia": 33.06,
        "Timisoara": 64.47
    },
    "Mehadia": {
        "Drobeta": 33.06,
        "Lugoj": 33.06
    },
    "Drobeta": {
        "Craiova": 72.69,
        "Mehadia": 33.06
    },
    "Sibiu": {
        "Fagaras": 82.3,
        "Rimnicu Vilcea": 45.18,
        "Oradea": 113.72,
        "Arad": 100.96
    },
    "Fagaras": {
        "Bucharest": 118.34,
        "Sibiu": 82.3
    },
    "Rimnicu Vilcea": {
        "Pitesti": 81.39,
        "Craiova": 103.25,
        "Sibiu": 45.18
    },
    "Craiova": {
        "Pitesti": 87.21,
        "Drobeta": 72.69
    },
    "Pitesti": {
        "Bucharest": 58.25,
        "Rimnicu Vilcea": 81.39
    },
    "Bucharest": {  # <----- Considerar Bucharest como la EMPRESA DE PAQUETERÍA
        "Giurgiu": 47.17,
        "Urziceni": 67.72,
        "Fagaras": 118.34
    },
    "Giurgiu": {
        "Bucharest": 47.17
    },
    "Urziceni": {
        "Hirsova": 63.0,
        "Vaslui": 89.94,
        "Bucharest": 67.72
    },
    "Hirsova": {
        "Eforie": 51.88,
        "Urziceni": 63.0
    },
    "Eforie": {
        "Hirsova": 51.88
    },
    "Vaslui": {
        "Iasi": 58.67,
        "Urziceni": 89.94
    },
    "Iasi": {
        "Neamt": 62.24,
        "Vaslui": 58.67
    },
    "Neamt": {
        "Iasi": 62.24
    }
}


In [162]:
ciudades = list(grafo.keys())
print(ciudades)

['Arad', 'Zerind', 'Oradea', 'Timisoara', 'Lugoj', 'Mehadia', 'Drobeta', 'Sibiu', 'Fagaras', 'Rimnicu Vilcea', 'Craiova', 'Pitesti', 'Bucharest', 'Giurgiu', 'Urziceni', 'Hirsova', 'Eforie', 'Vaslui', 'Iasi', 'Neamt']


In [163]:
import random

# Genera un paquete con peso, prioridad y destino aleatorios.
def generarPaquete():
    # Ejemplo de ciudades
    ciudades = list(grafo.keys())

    # Generar un paquete
    peso = random.randint(1, 50)  # Peso aleatorio del paquete
    prioridad = random.randint(1, 10)  # Prioridad aleatoria del 1 al 10, entre más mayor más prioridad
    nodo_entrega = random.choice(ciudades)  # Nodo de entrega aleatorio

    paquete = {
        "peso": peso,
        "prioridad": prioridad,
        "nodo_entrega": nodo_entrega
    }
    
    return paquete


In [164]:
generarPaquete()

{'peso': 23, 'prioridad': 1, 'nodo_entrega': 'Sibiu'}

In [165]:
# Genera una lista de 10 paquetes aleatorios.
# En este caso 10 paquetes, ya que el problema no especifica
# que el número de paquetes existentes deba ser aleatorio.
def generarListaPaquetes():
    lista_paquetes = []
    cantidad = 10
    for _ in range(cantidad):
        paquete = generarPaquete()
        lista_paquetes.append(paquete)
    return lista_paquetes

In [166]:
generarListaPaquetes()

[{'peso': 27, 'prioridad': 7, 'nodo_entrega': 'Zerind'},
 {'peso': 13, 'prioridad': 4, 'nodo_entrega': 'Bucharest'},
 {'peso': 5, 'prioridad': 6, 'nodo_entrega': 'Mehadia'},
 {'peso': 9, 'prioridad': 3, 'nodo_entrega': 'Sibiu'},
 {'peso': 47, 'prioridad': 2, 'nodo_entrega': 'Giurgiu'},
 {'peso': 39, 'prioridad': 1, 'nodo_entrega': 'Neamt'},
 {'peso': 7, 'prioridad': 6, 'nodo_entrega': 'Craiova'},
 {'peso': 33, 'prioridad': 7, 'nodo_entrega': 'Drobeta'},
 {'peso': 39, 'prioridad': 8, 'nodo_entrega': 'Drobeta'},
 {'peso': 50, 'prioridad': 6, 'nodo_entrega': 'Mehadia'}]

In [167]:
def generarRepartidor(lista_paquetes):
    # Definir la capacidad de carga aleatoria del repartidor
    capacidad_carga = random.randint(50, 150)  # Peso máximo que puede llevar

    # Ubicación inicial del repartidor, que es la ciudad de la empresa
    ubicacion_inicial = "Bucharest"

    # Inicializar la lista de paquetes que puede llevar
    paquetes = []
    peso_total = 0

    # Seleccionar paquetes que el repartidor puede llevar sin exceder su capacidad
    for paquete in lista_paquetes[:]:  # Hacemos una copia de la lista para evitar problemas al modificarla
        if peso_total + paquete['peso'] <= capacidad_carga:
            paquetes.append(paquete)
            peso_total += paquete['peso']
            lista_paquetes.remove(paquete)  # Remover el paquete de la lista original

    # Obtener la ruta a partir de los nodos de entrega de los paquetes
    ruta = [paquete['nodo_entrega'] for paquete in paquetes]

    # Crear el repartidor como un diccionario
    repartidor = {
        "capacidad": capacidad_carga,
        "ubicacion": ubicacion_inicial,
        "carga": paquetes,  # Campo para almacenar los paquetes
        "peso_total": peso_total,
        "ruta": ruta    # Lista de ciudades de entrega
    }

    return repartidor

In [168]:
# Prueba para comparar que el repartidor tome correctamente los paquetes
# en base a la capacidad y que éstos se salgan de la lista de paquetes.
test = generarListaPaquetes()
print(test)
test2 = generarRepartidor(test)
print(test2)
print(test)

[{'peso': 27, 'prioridad': 8, 'nodo_entrega': 'Mehadia'}, {'peso': 34, 'prioridad': 9, 'nodo_entrega': 'Urziceni'}, {'peso': 40, 'prioridad': 3, 'nodo_entrega': 'Vaslui'}, {'peso': 10, 'prioridad': 8, 'nodo_entrega': 'Mehadia'}, {'peso': 47, 'prioridad': 5, 'nodo_entrega': 'Vaslui'}, {'peso': 24, 'prioridad': 9, 'nodo_entrega': 'Giurgiu'}, {'peso': 19, 'prioridad': 8, 'nodo_entrega': 'Lugoj'}, {'peso': 7, 'prioridad': 1, 'nodo_entrega': 'Arad'}, {'peso': 20, 'prioridad': 4, 'nodo_entrega': 'Pitesti'}, {'peso': 32, 'prioridad': 2, 'nodo_entrega': 'Oradea'}]
{'capacidad': 150, 'ubicacion': 'Bucharest', 'carga': [{'peso': 27, 'prioridad': 8, 'nodo_entrega': 'Mehadia'}, {'peso': 34, 'prioridad': 9, 'nodo_entrega': 'Urziceni'}, {'peso': 40, 'prioridad': 3, 'nodo_entrega': 'Vaslui'}, {'peso': 10, 'prioridad': 8, 'nodo_entrega': 'Mehadia'}, {'peso': 24, 'prioridad': 9, 'nodo_entrega': 'Giurgiu'}, {'peso': 7, 'prioridad': 1, 'nodo_entrega': 'Arad'}], 'peso_total': 142, 'ruta': ['Mehadia', 'Urz

In [169]:
def reconstruir_camino(camino, ciudad_actual):
    ruta_total = [ciudad_actual]
    while ciudad_actual in camino:
        ciudad_actual = camino[ciudad_actual]
        ruta_total.append(ciudad_actual)
    return ruta_total[::-1]  # Retornar la ruta desde el inicio hasta el destino

In [170]:
import heapq

def a_star(ciudad_inicial, ciudad_objetivo, grafo):
    # Crear una lista de prioridades (min-heap)
    conjunto_abierto = []
    heapq.heappush(conjunto_abierto, (0, ciudad_inicial))  # (costo total, ciudad actual)

    camino = {}  # Almacena el camino desde el inicio hasta cada ciudad
    costo_real = {ciudad: float('inf') for ciudad in grafo}  # Costo desde el inicio
    costo_real[ciudad_inicial] = 0

    costo_estimado_total = {ciudad: float('inf') for ciudad in grafo}  # Costo estimado total
    costo_estimado_total[ciudad_inicial] = distancia_euclidiana(coordenadas[ciudad_inicial], coordenadas[ciudad_objetivo])

    while conjunto_abierto:
        costo_total_actual, ciudad_actual = heapq.heappop(conjunto_abierto)

        if ciudad_actual == ciudad_objetivo:
            return reconstruir_camino(camino, ciudad_actual)

        for ciudad_vecina, distancia_hacia_vecina in grafo[ciudad_actual].items():
            costo_tentativo = costo_real[ciudad_actual] + distancia_hacia_vecina

            if costo_tentativo < costo_real[ciudad_vecina]:
                camino[ciudad_vecina] = ciudad_actual
                costo_real[ciudad_vecina] = costo_tentativo
                costo_estimado_total[ciudad_vecina] = costo_tentativo + distancia_euclidiana(coordenadas[ciudad_vecina], coordenadas[ciudad_objetivo])

                if (costo_tentativo, ciudad_vecina) not in conjunto_abierto:
                    heapq.heappush(conjunto_abierto, (costo_estimado_total[ciudad_vecina], ciudad_vecina))

    return []  # Retornar una lista vacía si no se encuentra ruta


In [171]:
def calcular_recorrido(repartidor, grafo):
    # Iniciar en la ubicación inicial del repartidor
    ubicacion_actual = repartidor["ubicacion"]
    ruta_completa = []

    # Para cada paquete en la carga del repartidor
    for paquete in repartidor["carga"]:
        destino = paquete['nodo_entrega']
        
        # Calcular la ruta desde la ubicación actual a la ciudad de entrega
        ruta_a_destino = a_star(ubicacion_actual, destino, grafo)
        
        # Añadir la ruta a la ruta completa, evitando duplicados
        for ciudad in ruta_a_destino[:-1]:  # Hasta el penúltimo para evitar duplicar el destino
            if not ruta_completa or ciudad != ruta_completa[-1]:  # Evitar duplicados
                ruta_completa.append(ciudad)

        # Añadir el destino actual si no es igual a la última ciudad en la ruta
        if not ruta_completa or destino != ruta_completa[-1]:
            ruta_completa.append(destino)  # Añadir el destino actual

        # Actualizar la ubicación actual
        ubicacion_actual = destino

    # Finalmente, regresar a Bucharest
    ruta_a_empresa = a_star(ubicacion_actual, "Bucharest", grafo)
    
    # Añadir la ruta de regreso, evitando duplicados
    for ciudad in ruta_a_empresa:
        if not ruta_completa or ciudad != ruta_completa[-1]:  # Evitar duplicados
            ruta_completa.append(ciudad)

    # Actualizar la ruta del repartidor
    repartidor["ruta"] = ruta_completa
    
    # Imprimir el recorrido completo
    print("Recorrido completo del repartidor:", " -> ".join(ruta_completa))
    
    return ruta_completa


In [172]:
def calcular_costo_recorrido(repartidor, grafo):
    costo_total = 0
    ubicacion_actual = repartidor['ubicacion']

    # Para cada ciudad en la ruta del repartidor
    for ciudad_destino in repartidor['ruta']:
        # Obtener la distancia al destino
        #print("test1 = {}".format(ciudad_destino))
        if ciudad_destino in grafo[ubicacion_actual]:
            distancia = grafo[ubicacion_actual][ciudad_destino]
            costo_total += distancia  # Sumar la distancia al costo total
            #print("test2 = {}".format(distancia))
            ubicacion_actual = ciudad_destino  # Actualizar la ubicación actual

    return costo_total

In [173]:
# Generar una lista de paquetes
lista_paquetes = generarListaPaquetes()
print("lista = {}".format(lista_paquetes))

# Generar el repartidor con los paquetes que puede llevar
repartidor = generarRepartidor(lista_paquetes)
print("repartidor = {}".format(repartidor))

# Calcular y mostrar el recorrido
calcular_recorrido(repartidor, grafo)

# Calcular el costo del recorrido del repartidor
costo = calcular_costo_recorrido(repartidor, grafo)
print(f"Costo total del recorrido: {costo}")


lista = [{'peso': 36, 'prioridad': 5, 'nodo_entrega': 'Giurgiu'}, {'peso': 3, 'prioridad': 4, 'nodo_entrega': 'Fagaras'}, {'peso': 47, 'prioridad': 10, 'nodo_entrega': 'Hirsova'}, {'peso': 42, 'prioridad': 5, 'nodo_entrega': 'Mehadia'}, {'peso': 4, 'prioridad': 9, 'nodo_entrega': 'Sibiu'}, {'peso': 3, 'prioridad': 7, 'nodo_entrega': 'Drobeta'}, {'peso': 39, 'prioridad': 6, 'nodo_entrega': 'Timisoara'}, {'peso': 5, 'prioridad': 6, 'nodo_entrega': 'Drobeta'}, {'peso': 21, 'prioridad': 8, 'nodo_entrega': 'Arad'}, {'peso': 7, 'prioridad': 2, 'nodo_entrega': 'Zerind'}]
repartidor = {'capacidad': 54, 'ubicacion': 'Bucharest', 'carga': [{'peso': 36, 'prioridad': 5, 'nodo_entrega': 'Giurgiu'}, {'peso': 3, 'prioridad': 4, 'nodo_entrega': 'Fagaras'}, {'peso': 4, 'prioridad': 9, 'nodo_entrega': 'Sibiu'}, {'peso': 3, 'prioridad': 7, 'nodo_entrega': 'Drobeta'}, {'peso': 5, 'prioridad': 6, 'nodo_entrega': 'Drobeta'}], 'peso_total': 51, 'ruta': ['Giurgiu', 'Fagaras', 'Sibiu', 'Drobeta', 'Drobeta']}
R

In [174]:
def calcular_probabilidad_entrega(paquete, lista_paquetes):
    # Sumar las prioridades de todos los paquetes
    suma_prioridades = sum(p['prioridad'] for p in lista_paquetes)
    
    # Calcular la probabilidad
    probabilidad = paquete['prioridad'] / suma_prioridades if suma_prioridades > 0 else 0
    return probabilidad


In [175]:
def convertir_a_individuo(repartidor):
    calcular_recorrido(repartidor,grafo)

    # Crear el individuo con la carga y el costo
    individuo = {
        'carga': repartidor['carga'],
        'costo_total': calcular_costo_recorrido(repartidor,grafo)
    }
    return individuo

In [181]:
def calcular_fitness(individuo):
    # Obtener el costo total del recorrido
    costo_total = individuo['costo_total']
    
    # Calcular la prioridad acumulada basada en la probabilidad de entrega
    prioridad_total = 0
    for paquete in individuo['carga']:
        prioridad_total += calcular_probabilidad_entrega(paquete, individuo['carga'])
    
    # Invertimos el costo para que el fitness sea mayor para recorridos más cortos
    fitness = (prioridad_total * 100) / (1 + costo_total)  # Escalar fitness para favorecer menor costo

    return fitness


In [182]:
repartidor = generarRepartidor(generarListaPaquetes())
print("repartidor = {}".format(repartidor))

lista_paquetes = repartidor['carga']
print(lista_paquetes[0])

prioridad = calcular_probabilidad_entrega(lista_paquetes[0], lista_paquetes)
print("prioridad = {}".format(prioridad))

individuo = convertir_a_individuo(repartidor)  # Convertimos la carga en un individuo
print(individuo)

fitness_valor = calcular_fitness(individuo)
print("Valor de fitness del individuo:", fitness_valor)


repartidor = {'capacidad': 81, 'ubicacion': 'Bucharest', 'carga': [{'peso': 11, 'prioridad': 5, 'nodo_entrega': 'Vaslui'}, {'peso': 10, 'prioridad': 10, 'nodo_entrega': 'Giurgiu'}, {'peso': 9, 'prioridad': 6, 'nodo_entrega': 'Giurgiu'}, {'peso': 7, 'prioridad': 2, 'nodo_entrega': 'Hirsova'}, {'peso': 7, 'prioridad': 1, 'nodo_entrega': 'Pitesti'}, {'peso': 30, 'prioridad': 3, 'nodo_entrega': 'Vaslui'}], 'peso_total': 74, 'ruta': ['Vaslui', 'Giurgiu', 'Giurgiu', 'Hirsova', 'Pitesti', 'Vaslui']}
{'peso': 11, 'prioridad': 5, 'nodo_entrega': 'Vaslui'}
prioridad = 0.18518518518518517
Recorrido completo del repartidor: Bucharest -> Urziceni -> Vaslui -> Urziceni -> Bucharest -> Giurgiu -> Bucharest -> Urziceni -> Hirsova -> Urziceni -> Bucharest -> Fagaras -> Sibiu -> Rimnicu Vilcea -> Pitesti -> Bucharest -> Urziceni -> Vaslui -> Urziceni -> Bucharest
{'carga': [{'peso': 11, 'prioridad': 5, 'nodo_entrega': 'Vaslui'}, {'peso': 10, 'prioridad': 10, 'nodo_entrega': 'Giurgiu'}, {'peso': 9, 'prio